In [1]:
# from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
# from pymoo.operators.sampling.rnd import BinaryRandomSampling
# from pymoo.operators.crossover.hux import HUX
# from pymoo.algorithms.moo.nsga2 import NSGA2
# from pymoo.indicators.hv import Hypervolume
# from pymoo.core.mutation import Mutation
# from pymoo.core.sampling import Sampling
# from pymoo.core.problem import Problem
# from pymoo.optimize import minimize

# from joblib import Parallel, delayed

# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch

# from scipy.stats import ranksums
# import matplotlib.pyplot as plt
# import torch.nn as nn

# import pandas as pd
# import numpy as np

# import dill

# import pickle
# import os
# import re

# from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import LabelEncoder

from main import *
from execution_schemes import *

datasets_path = "Datasets"
save_path = "_runs"

In [2]:
class GenericOptimizer(Problem):
	population_size = 100
	n_neighbours = 5
	sequential = False
	def __init__(self, X_train, y_train, X_val, y_val, objectives, exec_mode):
		self.mutation_history = {}
		self.generation_number = 0

		self.exec_mode = exec_mode

		self.X_train = X_train
		self.y_train = y_train

		self.X_val = X_val
		self.y_val = y_val

		self.training_data = X_train
		self.n_instances = X_train.shape[0]
		
		self.objectives = objectives

		super().__init__(
			n_var=self.n_instances,
			n_obj=len(objectives),               
			n_constr=0,            
			xl=0,                  
			xu=1,                  
			type_var=np.bool_,     
		)

	def _evaluate(self, x, out, *args, **kwargs):
		
		if self.exec_mode == "sequential":
			metrics = []
			for objective in self.objectives:
				metrics.append(self.eval_objective((objective, x)))
		else:
			metrics = Parallel(n_jobs=-1)(delayed(self.eval_objective)((objective, x)) for objective in self.objectives)
		
		self.generation_number += 1

		out["F"] = np.column_stack(metrics)

	def eval_objective(self, pack):
		objective, x = pack
			
		if "calculate_num_examples" in repr(objective):
			return GenericOptimizer.calculate_num_examples(x)

		elif "calculate_IR" in repr(objective):
			vals = []
			for instance in x:
				vals.append(GenericOptimizer.calculate_IR(self.y_train[instance]))
			return vals
		
		else:
			vals = []
			for instance in x:
				vals.append(objective(
					self.X_train[instance],
					self.y_train[instance],
					self.X_val,
					self.y_val,
					GenericOptimizer.n_neighbours
				))
			return vals

	@classmethod
	def calculate_IR(cls, y):
		df = pd.DataFrame(y).value_counts()
		return (df[1]/df[0]) if df.min() == 0 else (df[0]/df[1])
	
	@classmethod
	def filter_by_class(cls, x, y, label):
		indices = np.where(y==label)
		return x[indices], y[indices]
	
	@classmethod
	def calculate_overall_error(cls, x_train, y_train, x_val, y_val, n):
				
		num_included_instances = x_train.shape[0]

		if num_included_instances >= n:
			optimization_knn = KNeighborsClassifier(n_neighbors=n)
			optimization_knn.fit(x_train, y_train)

			y_pred = optimization_knn.predict(x_val)
			acc = accuracy_score(y_val, y_pred)
			return 1-acc
		else:
			return 1

	@classmethod
	def calculate_class0_error(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 0)
		err = cls.calculate_overall_error(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return err

	@classmethod
	def calculate_class1_error(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 1)
		err = cls.calculate_overall_error(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return err

	@classmethod
	def calculate_overall_inverse_f1(cls, x_train, y_train, x_val, y_val, n):
				
		num_included_instances = x_train.shape[0]

		if num_included_instances >= n:
			optimization_knn = KNeighborsClassifier(n_neighbors=n)
			optimization_knn.fit(x_train, y_train)

			y_pred = optimization_knn.predict(x_val)
			f1 = f1_score(y_val, y_pred)
			return 1-f1
		else:
			return 1

	@classmethod
	def calculate_class0_inverse_f1(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 0)
		inv_f1 = cls.calculate_overall_inverse_f1(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return inv_f1

	@classmethod
	def calculate_class1_inverse_f1(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 1)
		inv_f1 = cls.calculate_overall_inverse_f1(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return inv_f1
	
	@classmethod
	def calculate_overall_inverse_precision(cls, x_train, y_train, x_val, y_val, n):
				
		num_included_instances = x_train.shape[0]

		if num_included_instances >= n:
			optimization_knn = KNeighborsClassifier(n_neighbors=n)
			optimization_knn.fit(x_train, y_train)

			y_pred = optimization_knn.predict(x_val)
			prec = precision_score(y_val, y_pred)
			return 1-prec
		else:
			return 1

	@classmethod
	def calculate_class0_inverse_precision(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 0)
		inv_prec = cls.calculate_overall_inverse_precision(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return inv_prec

	@classmethod
	def calculate_class1_inverse_precision(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 1)
		inv_prec = cls.calculate_overall_inverse_precision(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return inv_prec
		
	@classmethod
	def calculate_overall_inverse_recall(cls, x_train, y_train, x_val, y_val, n):
				
		num_included_instances = x_train.shape[0]

		if num_included_instances >= n:
			optimization_knn = KNeighborsClassifier(n_neighbors=n)
			optimization_knn.fit(x_train, y_train)

			y_pred = optimization_knn.predict(x_val)
			recall = recall_score(y_val, y_pred)
			return 1-recall
		else:
			return 1

	@classmethod
	def calculate_class0_inverse_recall(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 0)
		inv_recall = cls.calculate_overall_inverse_recall(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return inv_recall

	@classmethod
	def calculate_class1_inverse_recall(cls, x_train, y_train, x_val, y_val, n):
		class0_x_train, class0_y_train = cls.filter_by_class(x_train, y_train, 1)
		inv_recall = cls.calculate_overall_inverse_recall(
			class0_x_train,
			class0_y_train,
			x_val,
			y_val,
			n
		)
		return inv_recall
	
	@classmethod
	def calculate_num_examples(cls, instances):
		return np.sum(instances, axis=1)

	@classmethod
	def quantify_performance(cls, population, objectives, x_train, y_train, x_validation, y_validation, x_test, y_test):
		pass

	@classmethod
	def unbound_eval_objectives(cls, objective, instances, x_train, y_train, x_validation, y_validation):
		if "calculate_num_examples" in repr(objective):
			return GenericOptimizer.calculate_num_examples(instances)

		elif "calculate_IR" in repr(objective):
			vals = []
			for instance in instances:
				vals.append(GenericOptimizer.calculate_IR(y_train[instance]))
			return vals
		
		else:
			vals = []
			for instance in instances:
				vals.append(objective(
					x_train[instance],
					y_train[instance],
					x_validation,
					y_validation,
					GenericOptimizer.n_neighbours
				))
			return vals
		
	@classmethod
	def calculate_optimal_instance(cls, x_train, y_train, x_val, y_val, metrics, population, n):

		fronts = NonDominatedSorting().do(metrics, only_non_dominated_front=True)
		_, pareto_indicies = np.unique(metrics[fronts], axis=0, return_index=True)

		best_acc = 0
		best_instance = None
		for idx, instance in enumerate(population[pareto_indicies]):
			x_filtered, y_filtered = x_train[instance], y_train[instance]
			if x_filtered.shape[0] < n: 
				acc = 0
			else:
				knn = KNeighborsClassifier(n_neighbors=n)
				knn.fit(x_filtered, y_filtered)
				y_pred = knn.predict(x_val)
				acc = accuracy_score(y_val, y_pred)
			
				if acc > best_acc:
					best_acc = acc
					best_instance = instance
				
		return pareto_indicies, x_train[best_instance], y_train[best_instance]
	  
def prepare_splits(path):
	try:
		df = pd.read_csv(path, delimiter=', ', engine='python')
		x = df.drop(columns='Class')
		y = df['Class']
	except KeyError:
		df = pd.read_csv(path, delimiter=',')
		x = df.drop(columns='Class')
		y = df['Class']

	x = np.array(x)
	
	label_encoder = LabelEncoder()
	y = label_encoder.fit_transform(y)
	
	train_split = StratifiedShuffleSplit(
		n_splits=31,
		test_size=0.5,
	)

	splits = []

	for train_idx, temp_idx in train_split.split(x, y):

		test_split = StratifiedShuffleSplit(
			n_splits=1,
			test_size=0.5
		)

		x_temp, y_temp = x[temp_idx], y[temp_idx]

		test_idx, validation_idx = next(test_split.split(x_temp, y_temp))

		validation_idx = temp_idx[validation_idx]
		test_idx = temp_idx[test_idx]

		splits.append((train_idx, validation_idx, test_idx))
	
	return x, y, splits

def over_sample(x, y):
	
	counts = pd.DataFrame(y).value_counts()

	if counts[0] < counts[1]:
		minority_class_indicies = np.where(y == 1)
	else:
		minority_class_indicies = np.where(y == 0)

	over_sampled_x = np.concatenate((x, x[minority_class_indicies]), axis=0)
	over_sampled_y = np.concatenate((y, y[minority_class_indicies]), axis=0)

	return over_sampled_x, over_sampled_y

class CustomDataset(Dataset):
	def __init__(self, x_train, y_train):
		self.x_train = x_train
		self.y_train = y_train
	def __len__(self):
		return self.x_train.shape[0]
	def __getitem__(self, ind):
		x = self.x_train[ind]
		y = self.y_train[ind]
		return x, y

class MLP(nn.Module):
	def __init__(self, input_dim):
		super(MLP, self).__init__()
		self.linear1 = nn.Linear(input_dim, input_dim//2)
		self.relu1 = nn.ReLU()
		self.linear2 = nn.Linear(input_dim//2, input_dim//3)
		self.relu2 = nn.ReLU()
		self.linear3 = nn.Linear(input_dim//3, input_dim)

	def forward(self, x):
		x = self.linear1(x)
		x = self.relu1(x)
		x = self.linear2(x)
		x = self.relu2(x)
		x = self.linear3(x)
		return x
	
class CustomMutation(Mutation):
	curr_MLP = None
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	num_synthetic_examples = 500000
	train_epochs = 50
	batch_size = 215
	primary_objective = GenericOptimizer.calculate_overall_error
	secondary_objectives = [
		[GenericOptimizer.calculate_num_examples],
		[GenericOptimizer.calculate_class0_error],
		[GenericOptimizer.calculate_class0_inverse_f1],
		[GenericOptimizer.calculate_class0_inverse_precision],
		[GenericOptimizer.calculate_class0_inverse_recall],
		[GenericOptimizer.calculate_class1_error],
		[GenericOptimizer.calculate_class1_inverse_f1],
		[GenericOptimizer.calculate_class1_inverse_precision],
		[GenericOptimizer.calculate_class1_inverse_recall],
		[GenericOptimizer.calculate_overall_inverse_f1],
		[GenericOptimizer.calculate_overall_inverse_precision],
		[GenericOptimizer.calculate_overall_inverse_recall],
		[GenericOptimizer.calculate_class0_inverse_precision, GenericOptimizer.calculate_class1_inverse_precision],
		[GenericOptimizer.calculate_class0_inverse_recall, GenericOptimizer.calculate_class1_inverse_recall],
		[GenericOptimizer.calculate_class0_inverse_f1, GenericOptimizer.calculate_class1_inverse_f1],
		[GenericOptimizer.calculate_class0_error, GenericOptimizer.calculate_class1_error],
	]

	def __init__(self, x_train, y_train, x_validation, y_validation, prediction_threshold=0.5):
		super().__init__()
		self.prediction_thresh = prediction_threshold
		synthesized_x, synthesized_y = CustomMutation.create_training_data(x_train, y_train, x_validation, y_validation)
		self.model = CustomMutation.train_mutation(synthesized_x, synthesized_y)

	def _do(self, problem, X, **kwargs):

		int_x = np.array(X, dtype=np.float32)
		dataset = CustomDataset(int_x, int_x)
		loader = DataLoader(dataset, batch_size=X.shape[0], shuffle=False)

		self.model.eval()
		with torch.no_grad():
			for data, _ in loader:
				data = data.to(CustomMutation.device)
				outputs = self.model(data)
				predictions = (outputs > self.prediction_thresh).bool()

		prediction = np.array(predictions)

		total_number_of_genes = X.shape[0] * X.shape[1]
		genes_effected = np.sum(X ^ prediction)

		if problem.generation_number not in problem.mutation_history:
			problem.mutation_history[problem.generation_number] = []
		
		problem.mutation_history[problem.generation_number].append(genes_effected/total_number_of_genes)
		return prediction

	@classmethod
	def train_mutation(cls, x_train, y_train):
		train_set = CustomDataset(x_train, y_train)
		input_dim = x_train.shape[1]
		batch_size = cls.batch_size
		train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

		device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

		 
		model = MLP(input_dim).to(device)
		optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
		criterion = nn.BCEWithLogitsLoss()

		model.train()
		for epoch in range(cls.train_epochs):
			losses = []
			for batch_num, input_data in enumerate(train_loader):
				optimizer.zero_grad()
				x, y = input_data
				x, y = x.to(device).float(), y.to(device)

				output = model(x)
				loss = criterion(output, y)
				loss.backward()
				losses.append(loss.item())
				optimizer.step()

		return model
	
	@classmethod
	def create_training_data(cls, x_train, y_train, x_validation, y_validation):
		
		synthesizing_splits = StratifiedShuffleSplit(
			n_splits=len(cls.secondary_objectives), # create a split for each secondary objective
			test_size=0.5, # Half the validation set is randomly excluded
		)	
		packages = []
		for idx, (sub_validation_idx, _) in enumerate(synthesizing_splits.split(x_validation, y_validation)):
			packages.append((
				cls.secondary_objectives[idx],
				x_train,
				y_train,
				x_validation[sub_validation_idx],
				y_validation[sub_validation_idx]
			))

		# Execute optimization and extract the final populations
		populations = Parallel(n_jobs=-1)(delayed(cls.execute_training_data_gen)(package) for package in packages)
		
		# Aggregate all populations into single list containing every unique instance
		all_instances = []
		for population in populations:
			for individual in population.pop:
				all_instances.append(list(individual.X))
				
		all_instances = np.array(all_instances)

		# Create synthetic examples by adding randin noise to each instance. Repeat until threshold is reached.
		synthetic_x, synthetic_y = [], []
		while len(synthetic_x) < cls.num_synthetic_examples:
			
			for y_true in all_instances:
				x_noised = []
				for idx, probability in enumerate(np.random.uniform(0.1, 1.0, y_true.shape[0])):
					if probability < 0.85:
						x_noised.append(y_true[idx])                
					else:
						x_noised.append(0 if y_true[idx] == 1 else 1)

				synthetic_x.append(np.array(x_noised, dtype=np.float32))
				synthetic_y.append(np.array(y_true, dtype=np.float32))

		return np.array(synthetic_x), np.array(synthetic_y)
	
	@classmethod
	def execute_training_data_gen(cls, package):

		objectives, x_train, y_train, x_validation, y_validation = package
		
		objectives.append(cls.primary_objective)

		problem = GenericOptimizer(
			x_train, 
			y_train, 
			x_validation, 
			y_validation,
			objectives,
			"Sequential"
		)

		algorithm = NSGA2(
			pop_size=GenericOptimizer.population_size, 
			sampling=BinaryRandomSampling(), 
			crossover=HUX(), 
			mutation=BitflipMutation(), 
			eliminate_duplicates=True
		)
		result = minimize(
			problem, 
			algorithm, 
			('n_gen', GenericOptimizer.population_size)
		)
		
		return result
	
class BitflipMutation(Mutation):

	def _do(self, problem, X, **kwargs):
		
		prob_var = self.get_prob_var(problem, size=(len(X), 1))
		Xp = np.copy(X)
		flip = np.random.random(X.shape) < prob_var
		Xp[flip] = ~X[flip]
		
		total_number_of_genes = X.shape[0] * X.shape[1]
		genes_effected = np.sum(X ^ Xp)

		if problem.generation_number not in problem.mutation_history:
			problem.mutation_history[problem.generation_number] = []
		
		problem.mutation_history[problem.generation_number].append(genes_effected/total_number_of_genes)

		return Xp

class BiasedBinarySampling(Sampling):
	def __init__(self, labels, major_prob, minor_prob):
		
		self.labels = labels
		counts = pd.DataFrame(labels).value_counts()
		if counts[0] > counts[1]:
			self.c0_thresh = major_prob
			self.c1_thresh = minor_prob
		else:
			self.c0_thresh = minor_prob
			self.c1_thresh = major_prob

		super().__init__()

	def _do(self, problem, n_samples, **kwargs):

		rands = np.random.random((n_samples, problem.n_var))
		init_pops = np.zeros((n_samples, problem.n_var), dtype=bool)
		for idx, label in enumerate(self.labels):
			if label == 0:
				init_pops[:, idx] = (rands[:, idx] < self.c0_thresh).astype(bool)
			if label == 1:
				init_pops[:, idx] = (rands[:, idx] < self.c1_thresh).astype(bool)


		return init_pops


# Legacy optimization

In [ ]:

evaluation_schemes = [
	[GenericOptimizer.calculate_overall_error, GenericOptimizer.calculate_num_examples],
	[GenericOptimizer.calculate_overall_error, GenericOptimizer.calculate_overall_inverse_f1, GenericOptimizer.calculate_num_examples],
	[GenericOptimizer.calculate_class0_error, GenericOptimizer.calculate_class1_error],
]

for objectives in evaluation_schemes:
	objectives_names = [re.search(r'\.([a-zA-Z_][a-zA-Z0-9_]*)\s', str(objective_name)).group(1) for objective_name in objectives]
	for folder in os.listdir(datasets_path):

		###########################################
		# overSample_randPop
		############################################
		x, y, splits = prepare_splits(os.path.join(datasets_path, folder, f"{folder}.csv"))
		packages = []
		for c, (train_idx, test_idx, validation_idx) in enumerate(splits): # 31 splits for Wilcoxon rank-sum
			packages.append((x, y, train_idx, validation_idx, test_idx, objectives, f"{c}_overSample_randPop_{folder}_{'_'.join(objectives_names)}"))

		print(f"\nCreated {len(packages)} packages")
		results = Parallel(n_jobs=-1)(delayed(overSample_randPop_execute)(package) for package in packages)
		print(f"\t- Finished {len(results)} optimizations")
		for result in results:
			with open(os.path.join(save_path, f"{result['name']}.pickle"), 'wb') as fh:
				pickle.dump(result, fh)	
				print(f"\t> Saved: {result['name']}")

		###########################################
		# regularSample_randPop
		############################################
		x, y, splits = prepare_splits(os.path.join(datasets_path, folder, f"{folder}.csv"))
		packages = []
		for c, (train_idx, test_idx, validation_idx) in enumerate(splits): # 31 splits for Wilcoxon rank-sum
			packages.append((x, y, train_idx, validation_idx, test_idx, objectives, f"{c}_regularSample_randPop_{folder}_{'_'.join(objectives_names)}"))

		print(f"\nCreated {len(packages)} packages")
		results = Parallel(n_jobs=-1)(delayed(regularSample_randPop_execute)(package) for package in packages)
		print(f"\t- Finished {len(results)} optimizations")
		for result in results:
			with open(os.path.join(save_path, f"{result['name']}.pickle"), 'wb') as fh:
				pickle.dump(result, fh)	
				print(f"\t> Saved: {result['name']}")

		###########################################
		# overSample_biasPop
		############################################
		x, y, splits = prepare_splits(os.path.join(datasets_path, folder, f"{folder}.csv"))
		packages = []
		for c, (train_idx, test_idx, validation_idx) in enumerate(splits): # 31 splits for Wilcoxon rank-sum
			packages.append((x, y, train_idx, validation_idx, test_idx, objectives, f"{c}_overSample_biasPop_{folder}_{'_'.join(objectives_names)}"))

		print(f"\nCreated {len(packages)} packages")
		results = Parallel(n_jobs=-1)(delayed(overSample_biasPop_execute)(package) for package in packages)
		print(f"\t- Finished {len(results)} optimizations")
		for result in results:
			with open(os.path.join(save_path, f"{result['name']}.pickle"), 'wb') as fh:
				pickle.dump(result, fh)	
				print(f"\t> Saved: {result['name']}")

		###########################################
		# regularSample_biasPop_execute
		############################################
		x, y, splits = prepare_splits(os.path.join(datasets_path, folder, f"{folder}.csv"))
		packages = []
		for c, (train_idx, test_idx, validation_idx) in enumerate(splits): # 31 splits for Wilcoxon rank-sum
			packages.append((x, y, train_idx, validation_idx, test_idx, objectives, f"{c}_regularSample_biasPop_{folder}_{'_'.join(objectives_names)}"))

		print(f"\nCreated {len(packages)} packages")
		results = Parallel(n_jobs=-1)(delayed(regularSample_biasPop_execute)(package) for package in packages)
		print(f"\t- Finished {len(results)} optimizations")
		for result in results:
			with open(os.path.join(save_path, f"{result['name']}.pickle"), 'wb') as fh:
				pickle.dump(result, fh)	
				print(f"\t> Saved: {result['name']}")


Created 31 packages
	- Finished 31 optimizations
	> Saved: 0_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 1_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 2_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 3_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 4_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 5_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 6_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 7_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 8_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 9_overSample_randPop_australian_calculate_overall_error_calculate_num_examples
	> Saved: 10_overSample_randPop_australian_calculate_overall

# Novel Mutator

In [ ]:
for folder in os.listdir(datasets_path):
	packages = []
	x, y, splits = prepare_splits(os.path.join(datasets_path, folder, f"{folder}.csv"))
	for c, (train_idx, test_idx, validation_idx) in enumerate(splits):
		packages.append((x, y, train_idx, validation_idx, f"{c}_autoEncoderMutation_{folder}" ))

	for package in packages:
		result = AutoEncoder_execute(package)
		with open(os.path.join(save_path, f"{result['name']}.pickle"), 'wb') as fh:
			pickle.dump(result, fh)	
			print(f"\t> Saved: {result['name']}")


C:\Users\RjKim\AppData\Local\Temp\ipykernel_33324\975327388.py:417: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  prediction = np.array(predictions)


	> Saved: 0_autoEncoderMutation_australian


C:\Users\RjKim\AppData\Local\Temp\ipykernel_33324\975327388.py:417: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  prediction = np.array(predictions)


	> Saved: 1_autoEncoderMutation_australian


C:\Users\RjKim\AppData\Local\Temp\ipykernel_33324\975327388.py:417: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  prediction = np.array(predictions)


...

	> Saved: 28_autoEncoderMutation_yeast1


C:\Users\RjKim\AppData\Local\Temp\ipykernel_33324\975327388.py:417: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  prediction = np.array(predictions)


	> Saved: 29_autoEncoderMutation_yeast1


C:\Users\RjKim\AppData\Local\Temp\ipykernel_33324\975327388.py:417: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  prediction = np.array(predictions)


	> Saved: 30_autoEncoderMutation_yeast1


# Out of distribution ideal mutation operator

In [12]:
class DistMutation(Mutation):
	curr_MLP = None
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	num_synthetic_examples = 500000
	train_epochs = 50
	batch_size = 215
	primary_objective = GenericOptimizer.calculate_overall_error
	secondary_objectives = [
		[GenericOptimizer.calculate_num_examples],
		[GenericOptimizer.calculate_class0_error],
		[GenericOptimizer.calculate_class0_inverse_f1],
		[GenericOptimizer.calculate_class0_inverse_precision],
		[GenericOptimizer.calculate_class0_inverse_recall],
		[GenericOptimizer.calculate_class1_error],
		[GenericOptimizer.calculate_class1_inverse_f1],
		[GenericOptimizer.calculate_class1_inverse_precision],
		[GenericOptimizer.calculate_class1_inverse_recall],
		[GenericOptimizer.calculate_overall_inverse_f1],
		[GenericOptimizer.calculate_overall_inverse_precision],
		[GenericOptimizer.calculate_overall_inverse_recall],
		[GenericOptimizer.calculate_class0_inverse_precision, GenericOptimizer.calculate_class1_inverse_precision],
		[GenericOptimizer.calculate_class0_inverse_recall, GenericOptimizer.calculate_class1_inverse_recall],
		[GenericOptimizer.calculate_class0_inverse_f1, GenericOptimizer.calculate_class1_inverse_f1],
		[GenericOptimizer.calculate_class0_error, GenericOptimizer.calculate_class1_error],
	]

	def __init__(self, x_train, y_train, x_validation, y_validation, prediction_threshold=0.5):
		super().__init__()
		self.prediction_thresh = prediction_threshold
		synthesized_x, synthesized_y = DistMutation.create_training_data(x_train, y_train, x_validation, y_validation)
		self.model = DistMutation.train_mutation(synthesized_x, synthesized_y)

	def _do(self, problem, X, **kwargs):

		int_x = np.array(X, dtype=np.float32)
		dataset = CustomDataset(int_x, int_x)
		loader = DataLoader(dataset, batch_size=X.shape[0], shuffle=False)

		self.model.eval()
		with torch.no_grad():
			for data, _ in loader:
				data = data.to(DistMutation.device)
				outputs = self.model(data)
				predictions = (outputs > self.prediction_thresh).bool()

		prediction = np.array(predictions)

		total_number_of_genes = X.shape[0] * X.shape[1]
		genes_effected = np.sum(X ^ prediction)

		if problem.generation_number not in problem.mutation_history:
			problem.mutation_history[problem.generation_number] = []
		
		problem.mutation_history[problem.generation_number].append(genes_effected/total_number_of_genes)
		return prediction

	@classmethod
	def train_mutation(cls, x_train, y_train):
		train_set = CustomDataset(x_train, y_train)
		input_dim = x_train.shape[1]
		batch_size = cls.batch_size
		train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

		device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

		 
		model = MLP(input_dim).to(device)
		optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
		criterion = nn.BCEWithLogitsLoss()

		model.train()
		for epoch in range(cls.train_epochs):
			losses = []
			for batch_num, input_data in enumerate(train_loader):
				optimizer.zero_grad()
				x, y = input_data
				x, y = x.to(device).float(), y.to(device)

				output = model(x)
				loss = criterion(output, y)
				loss.backward()
				losses.append(loss.item())
				optimizer.step()

		return model
	
	@classmethod
	def create_training_data(cls, x_train, y_train, x_validation, y_validation):
		
		synthesizing_splits = StratifiedShuffleSplit(
			n_splits=len(cls.secondary_objectives), # create a split for each secondary objective
			test_size=0.5, # Half the validation set is randomly excluded
		)	
		packages = []
		for idx, (synthesis_split, out_of_distribution_comparison_split) in enumerate(synthesizing_splits.split(x_validation, y_validation)):
			packages.append((
				cls.secondary_objectives[idx],
				x_train,
				y_train,
				x_validation[synthesis_split],
				y_validation[synthesis_split],
				synthesis_split,
				out_of_distribution_comparison_split
			))

		# Execute optimization and extract the final populations
		results = Parallel(n_jobs=-1)(delayed(cls.execute_training_data_gen)(package) for package in packages)
		
		# Aggregate all populations into single list containing every unique instance
		all_instances = []
		in_distribution_error_vals = []
		out_of_distribution_error_vals = []

		for result, synthesis_split, comparison_split in results:
			x_in_distribution = x_validation[synthesis_split]
			y_in_distribution = y_validation[synthesis_split]

			x_out_of_distribution = x_validation[comparison_split]
			y_out_of_distribution = y_validation[comparison_split]

			for problem in result.history:
				for individual in problem.pop:
					instance = individual.X

					num_included_instances = x_train.shape[0]
					if num_included_instances >= GenericOptimizer.n_neighbours:
						optimization_knn = KNeighborsClassifier(n_neighbors=GenericOptimizer.n_neighbours)
						optimization_knn.fit(
							x_train[instance], 
							y_train[instance]
						)
						y_pred = optimization_knn.predict(x_in_distribution)
						acc = accuracy_score(y_in_distribution, y_pred)
						in_distribution_error = 1-acc

						y_pred = optimization_knn.predict(x_out_of_distribution)
						acc = accuracy_score(y_out_of_distribution, y_pred)
						out_of_distribution_error = 1-acc
					else:
						in_distribution_error = 1
						out_of_distribution_error = 1
					
					in_distribution_error_vals.append(in_distribution_error)
					out_of_distribution_error_vals.append(out_of_distribution_error)
					all_instances.append(instance)

		out_of_distribution_error_vals = np.array(out_of_distribution_error_vals)
		in_distribution_error_vals = np.array(in_distribution_error_vals)
		all_instances = np.array(all_instances)

		# sorted_idx_in_dist = np.argsort(out_of_distribution_error_vals)[:3000]
		sorted_idx_out_dist = np.argsort(in_distribution_error_vals)[:6000]

		# select_indices = list(set(np.concatenate((sorted_idx_in_dist, sorted_idx_out_dist))))
		# selected_instances = all_instances[select_indices]

		selected_instances = all_instances[sorted_idx_out_dist]

		# Create synthetic examples by adding randin noise to each instance. Repeat until threshold is reached.
		synthetic_x, synthetic_y = [], []
		while len(synthetic_x) < cls.num_synthetic_examples:
			
			for y_true in selected_instances:
				x_noised = []
				for idx, probability in enumerate(np.random.uniform(0.1, 1.0, y_true.shape[0])):
					if probability < 0.85:
						x_noised.append(y_true[idx])                
					else:
						x_noised.append(0 if y_true[idx] == 1 else 1)

				synthetic_x.append(np.array(x_noised, dtype=np.float32))
				synthetic_y.append(np.array(y_true, dtype=np.float32))

		return np.array(synthetic_x), np.array(synthetic_y)
	
	@classmethod
	def execute_training_data_gen(cls, package):

		objectives, x_train, y_train, x_validation, y_validation, validation_split, test_split = package
		
		objectives.append(cls.primary_objective)

		problem = GenericOptimizer(
			x_train, 
			y_train, 
			x_validation, 
			y_validation,
			objectives,
			"Sequential"
		)

		algorithm = NSGA2(
			pop_size=GenericOptimizer.population_size, 
			sampling=BinaryRandomSampling(), 
			crossover=HUX(), 
			mutation=BitflipMutation(), 
			eliminate_duplicates=True
		)
		result = minimize(
			problem, 
			algorithm, 
			('n_gen', GenericOptimizer.population_size),
			save_history=True
		)
		
		return result, validation_split, test_split
	
def OutOfDist_execute(package):
	x, y, train_idx, validation_idx, run_name = package
	x_train, y_train = x[train_idx], y[train_idx]
	x_validation, y_validation = x[validation_idx], y[validation_idx]

	mutation_operator = DistMutation(
		x_train, y_train, 
		x_validation, y_validation,
		prediction_threshold=0.5
	)

	algorithm = NSGA2(
		pop_size=GenericOptimizer.population_size, 
		sampling=BinaryRandomSampling(), 
		crossover=HUX(), 
		mutation=mutation_operator, 
		eliminate_duplicates=True
	)
	problem = GenericOptimizer(
		x_train, 
		y_train, 
		x_validation, 
		y_validation,
		[CustomMutation.primary_objective, GenericOptimizer.calculate_num_examples],
		"Sequential"
	)
	result = minimize(
		problem, 
		algorithm, 
		('n_gen', GenericOptimizer.population_size),
		verbose=False,
		save_history=True
	)

	mutation_operator = None

	package = {
		"name": run_name,
		"train": train_idx,
		"validation": validation_idx,
		"test": test_idx,
		"result": result
	}
	return package

for folder in os.listdir(datasets_path):
	packages = []
	x, y, splits = prepare_splits(os.path.join(datasets_path, folder, f"{folder}.csv"))
	for c, (train_idx, test_idx, validation_idx) in enumerate(splits):
	
		result = OutOfDist_execute((x, y, train_idx, validation_idx, f"{c}_outOfDistMutation_{folder}" ))

		with open(os.path.join(save_path, f"{result['name']}.pickle"), 'wb') as fh:
			pickle.dump(result, fh)	
			print(f"\t> Saved: {result['name']}")

		result_dict = result
		train_idx = result_dict['train']
		validation_idx = result_dict['validation']
		result = result_dict['result']
		run_name = result_dict['name']

		if "autoEncoder" in run_name or "outOfDist" in run_name:
			dataset = run_name.split('_')[-1]
		else:
			print(run_name.split("_"))

		try:
			df = pd.read_csv(os.path.join(datasets_path, dataset, f'{dataset}.csv'), delimiter=', ', engine='python')
			x = df.drop(columns='Class')
			y = df['Class']
		except KeyError:
			df = pd.read_csv(os.path.join(datasets_path, dataset, f'{dataset}.csv'), delimiter=',')
			x = df.drop(columns='Class')
			y = df['Class']

		x = np.array(x)
		
		label_encoder = LabelEncoder()
		y = label_encoder.fit_transform(y)

		num_examples = x.shape[0]
		all_idx = np.arange(num_examples)
		prior_knowledge_idx = combined = np.union1d(train_idx, validation_idx)
		test_idx = np.setdiff1d(all_idx, prior_knowledge_idx)

		x_train, y_train = x[train_idx], y[train_idx]
		x_validation, y_validation = x[validation_idx], y[validation_idx]
		x_test, y_test = x[test_idx], y[test_idx]

		validation_pareto_front, best_x_train, best_y_train = GenericOptimizer.calculate_optimal_instance(
			x_train,
			y_train,
			x_validation,
			y_validation,
			result.F,
			result.X,
			GenericOptimizer.n_neighbours
		)
		reduction_rate = 1 - (best_x_train.shape[0] / x_train.shape[0])
		# optimized_ir = GenericOptimizer.calculate_IR(best_y_train)
		optimized_test_err = GenericOptimizer.calculate_overall_error(
			best_x_train,
			best_y_train,
			x_test,
			y_test,
			GenericOptimizer.n_neighbours
		)
		optimized_validation_err = GenericOptimizer.calculate_overall_error(
			best_x_train,
			best_y_train,
			x_validation,
			y_validation,
			GenericOptimizer.n_neighbours
		)
		baseline_test_err = GenericOptimizer.calculate_overall_error(
			x_train,
			y_train,
			x_test,
			y_test,
			GenericOptimizer.n_neighbours
		)
		baseline_validation_err = GenericOptimizer.calculate_overall_error(
			x_train,
			y_train,
			x_validation,
			y_validation,
			GenericOptimizer.n_neighbours
		)

		test_metrics = {}
		result.problem._evaluate(result.X, test_metrics)
		test_fitness = test_metrics['F']

		test_pareto_front, ideal_x_train, ideal_y_train = GenericOptimizer.calculate_optimal_instance(
			x_train,
			y_train,
			x_validation,
			y_validation,
			test_fitness,
			result.X,
			GenericOptimizer.n_neighbours
		)
		ideal_optimized_test_err = GenericOptimizer.calculate_overall_error(
			ideal_x_train,
			ideal_y_train,
			x_test,
			y_test,
			GenericOptimizer.n_neighbours
		)

		print(f"Baseline valdiation {1-baseline_validation_err}")
		print(f"Baseline test {1-baseline_test_err}")

		print(f"optimzed valdiation {1-optimized_validation_err}")
		print(f"optimzed test {1-optimized_test_err}")
		print(f"optimzed ideal {1-ideal_optimized_test_err}")

		break
	break

C:\Users\RjKim\AppData\Local\Temp\ipykernel_25796\2443781699.py:46: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  prediction = np.array(predictions)


	> Saved: 0_outOfDistMutation_australian
Baseline valdiation 0.6453488372093024
Baseline test 0.6473988439306358
optimzed valdiation 0.8081395348837209
optimzed test 0.6069364161849711
optimzed ideal 0.6069364161849711
